In [1]:
from nsga2.estimator import NSGAIIRegressor
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv('../data/lexicase_paper/d_concrete.txt', sep=',')

# DEAP interface requires X and y to be numpy arrays, not pandas dataframes
X = df.drop('label', axis=1).values
y = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y)

estimator = NSGAIIRegressor(**{
    'pop_size'        : 80, 
    'max_gen'         : 100,
    'max_depth'       : 7,  # 8
    'max_size'        : 2**7, # 75
    'objectives'      : ['error', 'size'],
    'cx_prob'         : 1/5,
    'initialization'  : 'uniform',
    'pick_criteria'   : 'error', # error, MCDM
    'validation_size' : 0.33,
    'simplify'        : True,
    'simplification_method' : 'bottom_up',
    'simplification_tolerance' : 1e-2,
    'verbosity'       : 1,
    'survival'       : 'tournament',
    # 'functions'       :[
    #                 'div', 'add', 'sub', 'mul',
    #                 'add3', 'add4', 'mul3', 'mul4',
    #                 'maximum', 'minimum',
    #                 'sin', 'cos', 'tan', 'arcsin', 'arccos', 'arctan',
    #                 'log1p', 'expm1', 'log', 'exp', 
    #                 'sqrt', 'sqrtabs', 'square', 'abs' 
    #                 ]
}).fit(X_train, y_train)

gen	evals	best_size	n_simplifications	n_new_hashes	avg train error	avg train size	avg val error	avg val size	med train error	med train size	med val error	med val size	std train error	std train size	std val error	std val size	min train error	min train size	min val error	min val size	max train error	max train size	max val error	max val size
0  	80   	5        	141              	549         	               	              	             	            	               	              	             	            	               	              	             	            	               	              	             	            	               	              	             	            
1  	80   	1        	20               	92          	               	              	             	            	               	              	             	            	               	              	             	            	               	              	             	            	               	              	    

<string>:1: RuntimeWarning: invalid value encountered in sqrt


In [2]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse

model      = str(estimator.best_estimator_).replace("ARG", "x_")
size       = len(estimator.best_estimator_)
complexity = size
depth      = estimator.best_estimator_.height

print(model)
print(size)
print(complexity)
print(depth)

for metric, fn, (data_X, data_y) in [
    ('train_r2',  r2_score, (X_train, y_train)),
    ('test_r2',   r2_score, (X_test,  y_test )),
    ('train_mse', mse,      (X_train, y_train)),
    ('test_mse',  mse,      (X_test,  y_test )),
]:
    score = np.nan
    try:
        score = fn(estimator.predict(data_X), data_y)
        print(f"{metric} : {score}")
    except ValueError:
        print(f"(Failed to calculate {metric}")

for ind in estimator.archive_:
    print(ind.fitness, ind)

subtract(sqrt(mul4(x_0, sqrt(mul4(x_0, x_7, 0.04713249973666864, exp(arctan(x_1)))), 0.06520990666829518, exp(log1p(arctan(log1p(x_4)))))), -6.017489506550723)
19
19
7
train_r2 : 0.5964037870474304
test_r2 : 0.5911987224393906
train_mse : 76.89005091935277
test_mse : 100.1270186757325
(81.195209603625, 19.0) subtract(sqrt(mul4(ARG0, sqrt(mul4(ARG0, ARG7, 0.04713249973666864, exp(arctan(ARG1)))), 0.06520990666829518, exp(log1p(arctan(log1p(ARG4)))))), -6.017489506550723)
(inf, 5.0) subtract(sqrt(sin(ARG2)), -6.017489506550723)


In [3]:
print( len(list(estimator.simplifier.pop_hash.keys())) )

n_keys =  len(list(estimator.simplifier.pop_hash.keys()))

for key in list(estimator.simplifier.pop_hash.keys())[:n_keys]:
    print(key)
    for ind in estimator.simplifier.pop_hash[key]:
        print(" -", ind)

# 11001111011111110110111010010100
# 11010101000100110111001000101100

5144
0110011001001000001001001101100100101111011000000110100110100111110110110100000010010111000111100100111001110000110011001101000001011011101011100100001000001110001110010101100011101110000010000011011101000111010000000011100100001101100010001000100111000011
 - ARG0
 - absolute(ARG0)
 - minimum(ARG5, ARG0)
 - minimum(ARG0, ARG5)
 - log(expm1(ARG0))
 - expm1(log1p(ARG0))
 - minimum(ARG0, ARG0)
 - maximum(ARG4, ARG0)
 - exp(log(ARG0))
 - log1p(exp(ARG0))
 - minimum(ARG6, ARG0)
 - minimum(ARG0, ARG6)
 - maximum(0.0, ARG0)
 - maximum(ARG0, ARG0)
 - maximum(-45.58202271249168, ARG0)
 - maximum(0.04584242641423694, ARG0)
 - maximum(sqrtabs(ARG4), ARG0)
 - minimum(exp(ARG6), ARG0)
 - maximum(sqrt(mul4(0.0028731812020307097, ARG7, ARG0, ARG4)), ARG0)
0110001101001101101001001000100100011111011010000111100110110001110011110101110010000111100001101101110111010000100110011011001001000001101011001101111000011110101111001101111001111000101001111100001100100110110111110011100110000100110000001000

In [4]:
display(df.describe())

A = np.maximum(df['x0'], df['x4']+29.657).values
B = df['x6'].values

print(np.std(A) * (A - np.mean(A))[:5])
print(np.std(B) * (B - np.mean(B))[:5])

,x0,x1,x2,x3,x4,x5,x6,x7,label
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


[27036.47147454 27036.47147454  5361.93013146  5361.93013146
 -8624.67751114]
[ -7819.81227097  -7819.81227097 -14391.05039658 -14391.05039658
   4160.67675074]
